In [8]:
import concurrent.futures
import urllib
import json
import time
import os
import os.path
import pandas as pd
import re
import tornado.httpclient
from bs4 import BeautifulSoup

In [9]:
BASE_DIR = '/data/neon/customer_images/pinterest/10_categories'
#QUERIES = ['nursery', 'cake', 'woodworking', 'sailboat', 'travel', 'hiking', 'spring dress', 'spring', 
#           'italian recipes', 'dogs']
QUERIES = ['stir fry', 'container gardening', 'tattoos', 'superbowl', 'paleo', 'urban design',
           'abstract photography', 'halloween', 'makeup', 'hotrod']
API_TOKEN = 'AaBgobGn8YvYxWKnOX6ruHNdaW3_FHMOb9gvjGRDT8aQJsAr2QAAAAA'
BOARDS_PER_QUERY = 10
IMAGES_PER_BOARD = 200

In [10]:
def board_from_blogger(board):
    '''Returns True if the board is done by a blogger.
    
    Check the bio of the user and look for the word "blogger"
    '''
    url = 'https://api.pinterest.com/v1/users/%s/?access_token=%s&fields=bio' % (board.split('/')[0], API_TOKEN)
    data = json.loads(urllib.urlopen(url).read())
    return 'blog' in data['data']['bio'].lower()

In [11]:
def search_for_boards(query, max_boards=None, filter_func=None):
    '''Search for boards. Returns a list of board names'''
    url = 'https://www.pinterest.com/search/boards/?%s' % urllib.urlencode(
        {'q': unicode.encode(query, 'utf-8')})
    #'https://www.pinterest.com/resource/BaseSearchResource/get/?'
    #https://www.pinterest.com/resource/BoardsResource/get/? to get the list of boards for a user
    page = urllib.urlopen(url).read()
    soup = BeautifulSoup(page, 'html.parser')
    for script_tag in soup.find_all('script'):
        if script_tag.attrs.get('type') == 'application/json' and script_tag.attrs.get('id') == 'jsInit1':
            data = json.loads(script_tag.string)
            rv = [x['url'].strip('/') for x in data['tree']['data']['results']]
            if filter_func:
                rv = [x for x in rv if filter_func(x)]
            if max_boards:
                rv = rv[:max_boards]
            return rv
        
    raise IOError('Could not find boards for query %s' % query)


In [12]:
def list_user_boards(user):
    '''Returns a list of (board_id, name) of boards for a given user'''
    url = 'https://www.pinterest.com/resource/BoardsResource/get/?%s' % (urllib.urlencode(
        {'source_url' : '%s/boards/' % user,
         'data': json.dumps({
                'options' : {
                        'filter' : 'public',
                        'sort' : 'profile',
                        'field_set_key': 'grid_item',
                        'skip_board_create_rep': True,
                        'username': user
                },
                'context': {},
                
            }),
         '_': '1473961793194'
        }))
    client = tornado.httpclient.HTTPClient()
    headers = {
        'X-Requested-With': 'XMLHttpRequest',
        'Cache-Control':'no-cache',
        }
    response = client.fetch(tornado.httpclient.HTTPRequest(url, headers=headers))
    data = json.loads(response.body)
    return [(x['url'].strip('/'), x['name']) for x in data['resource_response']['data']]
    

In [13]:
def iterate_board_image_urls(board, max_images=None):
    url = 'https://api.pinterest.com/v1/boards/%s/pins/?cursor=&access_token=%s&fields=image' % (board, API_TOKEN)
    
    image_count = 0
    while url:
        data = json.loads(urllib.urlopen(url).read())
        url = data['page'].get('next')
        for entry in data['data']:
            if max_images and image_count >= max_images:
                return
            try:
                yield entry['image']['original']['url']
                image_count += 1
            except KeyError:
                pass


In [14]:
def process_board(query, board, base_dir):
    print 'Processing board %s' % board
    d = os.path.join(base_dir, query, board)
    if not os.path.exists(d):
        os.makedirs(d)
        
    for url in iterate_board_image_urls(board, max_images=IMAGES_PER_BOARD):
        fn = os.path.join(d, os.path.basename(url))
        if not os.path.exists(fn):
            urllib.urlretrieve(url, fn)

In [21]:
def process_query(query, base_dir=BASE_DIR, filter_func=board_from_blogger):
    for board in search_for_boards(unicode(query), BOARDS_PER_QUERY, filter_func):
        process_board(query, board, base_dir)


In [20]:
search_for_boards(unicode('hotrod'), 10, None)

[u'Ftsjp/hotrod',
 u'rockyhawkins/hotrod',
 u'tati878/hotrod',
 u'metinbilisli/hotrod',
 u'uglyrhino/hotrod',
 u'monstatruk/hotrod',
 u'areatattoostudi/hotrod',
 u'fukunglai/hotrod',
 u'stuartcollie/hotrod',
 u'danigon025/hotrod']

In [22]:
with concurrent.futures.ThreadPoolExecutor(6) as executor:
    futs = [executor.submit(process_query, x, filter_func=None) for x in QUERIES]
    concurrent.futures.wait(futs)

Processing board mooshujenne/superbowl
Processing board tattooblend/tattoos
Processing board inhabitat/urban-design
Processing board hometalk/container-gardening
Processing board dvcc/stir-fry
Processing board asiainferno/paleo
Processing board kalynskitchen/kalynskitchen-stir-fry-recipes
Processing board debo285/stir-fry
Processing board jennycookies/superbowl
Processing board hellogiggles/tattoos
Processing board GarlicandZest/superbowl
Processing board timmyt69/stir-fry
Processing board balconygardenwe/container-gardening
Processing board dai23i3/superbowl
Processing board igori/paleo
Processing board archigea/urban-design
Processing board adamdeming/tattoos
Processing board organiclifemag/container-gardening
Processing board mistilees/superbowl
Processing board OmnivorCookbook/stir-fry-recipes
Processing board douggreen/container-gardening
Processing board davidbromstad/tattoos
Processing board flower04/superbowl
Processing board urbndsgn/urban-design
Processing board sugarfreemom/

In [11]:
nurseries = ['bailey0722/nursery', 'plumprettysugar/nursery', 'smallable/nursery', 'spearmintbaby/nursery', 'designlovelife/nursery',
			'pinkblushmat/nursery', 'twinkandsis/nursery', 'bloginspire_/nursery', 'jujupapers/nursery', 'MommaSociety/nursery', 
			'littlegiraffexo/nursery', 'sarahakiyama/nursery', 'lilyjadeco/nursery', 'CasaBella1st/nursery', 'juneandjanuary_/nursery',
			'ThatStylishWife/nursery', 'Stonesegrist/nursery', 'gathrecompany/nursery', 'bravadogiftbox/nursery', 'rosandraferri/nursery',
			'abjane17/nursery', 'sarahlanedesign/nursery', 'markhed/nursery', 'cpavone/nursery', 'chelseabhansen/nursery', 'russelldouglas/nursery',
			'kalli86gl/nursery-room', 'mika_theo/nursery-rooms']

In [17]:
with concurrent.futures.ThreadPoolExecutor(6) as executor:
    futs = [executor.submit(process_board, 'nursery', x) for x in nurseries]
    concurrent.futures.wait(futs)

Processing board bailey0722/nursery
Processing board plumprettysugar/nursery
Processing board smallable/nursery
Processing board spearmintbaby/nursery
Processing board designlovelife/nursery
Processing board pinkblushmat/nursery
Processing board twinkandsis/nursery
Processing board bloginspire_/nursery
Processing board jujupapers/nursery
Processing board MommaSociety/nursery
Processing board littlegiraffexo/nursery
Processing board sarahakiyama/nursery
Processing board lilyjadeco/nursery
Processing board CasaBella1st/nursery
Processing board juneandjanuary_/nursery
Processing board ThatStylishWife/nursery
Processing board Stonesegrist/nursery
Processing board gathrecompany/nursery
Processing board bravadogiftbox/nursery
Processing board rosandraferri/nursery
Processing board abjane17/nursery
Processing board sarahlanedesign/nursery
Processing board markhed/nursery
Processing board cpavone/nursery
Processing board chelseabhansen/nursery
Processing board russelldouglas/nursery
Processing

In [66]:
cake_boards = search_for_boards('cake')
users = [x.split('/')[0] for x in cake_boards]
user_boards = {x: list_user_boards(x) for x in users}


In [76]:
boards = [{'user':user, 'board':x[0], 'name':x[1].lower()} for user, board_list in user_boards.iteritems() for x in board_list]
boards = pd.DataFrame(boards)
boards.groupby('name').count().sort_values('user', ascending=False)

,board,user
name,,
cake,37,37
easter,24,24
breakfast,18,18
cookies,17,17
christmas,17,17
thanksgiving,17,17
halloween,15,15
valentine's day,15,15
pasta,15,15


In [94]:
pair_boards = boards[(boards['name'] == 'cake') | (boards['name'] == 'bathroom')]
pair_boards = pair_boards.groupby('user').filter(lambda x: x.shape[0] > 1)


In [93]:
base_dir = '/data/neon/customer_images/pinterest/cross_reference_cake_bathroom'
with concurrent.futures.ThreadPoolExecutor(6) as executor:
    futs = [executor.submit(process_board, x['name'], x['board'], base_dir) for idx, x in pair_boards.iterrows()]
    concurrent.futures.wait(futs)

Processing board bohowedandlife/cake
Processing board bohowedandlife/bathroom
Processing board lambandblonde/bathroom
Processing board lambandblonde/cake
Processing board quebuenoesvivir/bathroom
Processing board quebuenoesvivir/cake
Processing board larapinea/cake
Processing board larapinea/bathroom
Processing board gatherandfeast/cake
Processing board gatherandfeast/bathroom
Processing board somethewiser/cake
Processing board somethewiser/bathroom
Processing board twopeasandpod/cake
Processing board twopeasandpod/bathroom
Processing board toshikouchino/bathroom
Processing board toshikouchino/cake
Processing board cysocki/bathroom
Processing board cysocki/cake
Processing board tempsec/cake
Processing board tempsec/bathroom
Processing board mrskamiller/cake
Processing board mrskamiller/bathroom


In [97]:
pair_boards = boards[(boards['name'] == 'cake') | (boards['name'] == 'cookies')]
pair_boards = pair_boards.groupby('user').filter(lambda x: x.shape[0] > 1)
base_dir = '/data/neon/customer_images/pinterest/cross_reference_cake_cookies'
with concurrent.futures.ThreadPoolExecutor(6) as executor:
    futs = [executor.submit(process_board, x['name'], x['board'], base_dir) for idx, x in pair_boards.iterrows()]
    concurrent.futures.wait(futs)

Processing board lambandblonde/cake
Processing board lambandblonde/cookies
Processing board hipfoodiemom1/cookies
Processing board hipfoodiemom1/cake
Processing board tastemade/cake
Processing board tastemade/cookies
Processing board larapinea/cookies
Processing board larapinea/cake
Processing board beauxoxo/cake
Processing board beauxoxo/cookies
Processing board suburbangourmet/cake
Processing board suburbangourmet/cookies
Processing board handleheat/cookies
Processing board handleheat/cake
Processing board somethewiser/cake
Processing board somethewiser/cookies
Processing board cmpollak1/cookies
Processing board cmpollak1/cake
Processing board femkedik/cake
Processing board femkedik/cookies
Processing board localhaven/cake
Processing board localhaven/cookies
Processing board twopeasandpod/cookies
Processing board twopeasandpod/cake
Processing board tidymom/cookies
Processing board Eatwell101/cookies
Processing board Eatwell101/cake
Processing board sprinklesomesug/cake
Processing boa

In [100]:
nursery_boards = search_for_boards('nursery')
nursery_users = [x.split('/')[0] for x in nursery_boards]
nursery_user_boards = {x: list_user_boards(x) for x in nursery_users}

In [101]:
nboards = [{'user':user, 'board':x[0], 'name':x[1].lower()} for user, board_list in nursery_user_boards.iteritems() for x in board_list]
nboards = pd.DataFrame(nboards)
nboards.groupby('name').count().sort_values('user', ascending=False)

,board,user
name,,
nursery,38,38
kitchen,16,16
diy,16,16
baby shower,10,10
bathroom,10,10
bedroom,10,10
art,9,9
christmas,8,8
easter,8,8


In [102]:
pair_boards = nboards[(nboards['name'] == 'nursery') | (nboards['name'] == 'kitchen')]
pair_boards = pair_boards.groupby('user').filter(lambda x: x.shape[0] > 1)
base_dir = '/data/neon/customer_images/pinterest/cross_reference_nursery_kitchen'
with concurrent.futures.ThreadPoolExecutor(6) as executor:
    futs = [executor.submit(process_board, x['name'], x['board'], base_dir) for idx, x in pair_boards.iterrows()]
    concurrent.futures.wait(futs)

Processing board sueprado/kitchen
Processing board sueprado/nursery
Processing board Foreverwildchld/nursery
Processing board Foreverwildchld/kitchen
Processing board jujupapers/kitchen
Processing board jujupapers/nursery
Processing board plumprettysugar/kitchen
Processing board plumprettysugar/nursery
Processing board jadephl/nursery
Processing board jadephl/kitchen
Processing board aptapothecary/kitchen
Processing board aptapothecary/nursery
Processing board turbulencesdeco/nursery
Processing board turbulencesdeco/kitchen
Processing board Stonesegrist/kitchen
Processing board Stonesegrist/nursery
Processing board spearmintbaby/kitchen
Processing board spearmintbaby/nursery
Processing board holycute1/kitchen
Processing board holycute1/nursery
Processing board apettyquarrel/kitchen
Processing board apettyquarrel/nursery
Processing board ninamoore/nursery
Processing board ninamoore/kitchen
Processing board DIYdecorideas/kitchen
Processing board DIYdecorideas/nursery
Processing board rut

In [16]:
fashion_queries = [u'fashion', u'mode', u'ファッション']
base_dir = '/data/neon/customer_images/pinterest/intl_fashion'
with concurrent.futures.ThreadPoolExecutor(6) as executor:
    boards = []
    for query in fashion_queries:
        boards.extend([(query, x) for x in search_for_boards(query, 30, None)])
            
    futs = [executor.submit(process_board, query, board, base_dir) for query, board in boards]
    concurrent.futures.wait(futs)
        

Processing board joannagoddard/fashion
Processing board codystonerock/fashion
Processing board bustledotcom/fashion
Processing board beaherzberg/fashion
Processing board marlienr/fashion
Processing board kholt/fashion
Processing board simplygrove/fashion
Processing board frenchbydesign/fashion
Processing board mdemulher/fashion
Processing board katemiss/fashion
Processing board weebirdy/fashion
Processing board dazedmagazine/fashion
Processing board aboutdotcom/fashion
Processing board theaestate/fashion
Processing board bailey0722/fashion
Processing board hexagone007/fashion
Processing board LuizaBo/fashion
Processing board thisisglamorous/fashion
Processing board milkmedia/fashion
Processing board designlovelife/fashion
Processing board stylowipl/fashion
Processing board twopeasandpod/fashion
Processing board thisispapermag/fashion
Processing board purewow/fashion
Processing board sundayselection/fashion
Processing board turquoise_erin/fashion
Processing board familycirclemag/fashion